# 4. Effect Size Meta-Analysis

This notebook guides you through:
1. Loading studies with effect sizes
2. Running random-effects meta-analysis
3. Assessing heterogeneity
4. Creating forest and funnel plots
5. Publication bias assessment

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

from core import Study, EffectSize, MetaAnalysisDataset
from analysis.effect_size import EffectSizeMetaAnalysis

## Step 1: Create Dataset with Effect Sizes

Effect sizes can be extracted from papers or computed from summary statistics.

In [ ]:
# Create dataset with example studies
dataset = MetaAnalysisDataset(
    name="T-Maze Performance Effects",
    description="Meta-analysis of intervention effects on T-maze performance"
)

# Example studies with effect sizes (Cohen's d)
studies_data = [
    {
        "study_id": "smith2020",
        "title": "Cognitive training improves spatial navigation",
        "authors": ["Smith, J.", "Jones, M."],
        "year": 2020,
        "n_treatment": 25,
        "n_control": 25,
        "effect_size": 0.65,
        "se": 0.29
    },
    {
        "study_id": "johnson2019",
        "title": "Exercise and spatial memory",
        "authors": ["Johnson, A."],
        "year": 2019,
        "n_treatment": 30,
        "n_control": 32,
        "effect_size": 0.42,
        "se": 0.26
    },
    {
        "study_id": "chen2021",
        "title": "Virtual reality training for navigation",
        "authors": ["Chen, L.", "Wang, X."],
        "year": 2021,
        "n_treatment": 40,
        "n_control": 38,
        "effect_size": 0.78,
        "se": 0.23
    },
    {
        "study_id": "garcia2022",
        "title": "Mindfulness and spatial cognition",
        "authors": ["Garcia, R."],
        "year": 2022,
        "n_treatment": 22,
        "n_control": 20,
        "effect_size": 0.31,
        "se": 0.31
    },
    {
        "study_id": "kim2023",
        "title": "Sleep and navigation performance",
        "authors": ["Kim, S.", "Lee, H."],
        "year": 2023,
        "n_treatment": 35,
        "n_control": 35,
        "effect_size": 0.55,
        "se": 0.24
    },
    {
        "study_id": "wilson2021",
        "title": "Neurofeedback and spatial skills",
        "authors": ["Wilson, P."],
        "year": 2021,
        "n_treatment": 28,
        "n_control": 30,
        "effect_size": 0.48,
        "se": 0.27
    },
    {
        "study_id": "taylor2020",
        "title": "Working memory training transfer",
        "authors": ["Taylor, M.", "Brown, K."],
        "year": 2020,
        "n_treatment": 45,
        "n_control": 42,
        "effect_size": 0.38,
        "se": 0.22
    },
    {
        "study_id": "martinez2022",
        "title": "Aerobic exercise and hippocampal function",
        "authors": ["Martinez, C."],
        "year": 2022,
        "n_treatment": 50,
        "n_control": 50,
        "effect_size": 0.62,
        "se": 0.20
    }
]

# Create studies with effect sizes
for data in studies_data:
    # Calculate variance from SE
    variance = data["se"] ** 2
    
    study = Study(
        study_id=data["study_id"],
        title=data["title"],
        authors=data["authors"],
        year=data["year"],
        n_total=data["n_treatment"] + data["n_control"],
        n_treatment=data["n_treatment"],
        n_control=data["n_control"],
        effect_sizes=[
            EffectSize(
                effect_size=data["effect_size"],
                variance=variance,
                effect_type="d",
                outcome_name="T-maze accuracy"
            )
        ]
    )
    dataset.add_study(study)

print(f"Created dataset with {dataset.n_studies} studies")
print(dataset.summary())

In [ ]:
# View effect sizes
es_df = dataset.to_effect_sizes_df()
es_df

## Step 2: Computing Effect Sizes from Raw Data

If papers report means and SDs instead of effect sizes, you can compute them.

In [ ]:
# Example: Computing Cohen's d from means and SDs
from core import EffectSize

# Raw data from a hypothetical paper
mean_treatment = 85.2
sd_treatment = 12.4
n_treatment = 30

mean_control = 78.5
sd_control = 11.8
n_control = 28

# Use the from_means_sds class method
computed_es = EffectSize.from_means_sds(
    mean_exp=mean_treatment,
    sd_exp=sd_treatment,
    n_exp=n_treatment,
    mean_ctrl=mean_control,
    sd_ctrl=sd_control,
    n_ctrl=n_control,
    outcome_name="Computed example"
)

print(f"Computed effect size (Cohen's d): {computed_es.effect_size:.3f}")
print(f"Standard error: {np.sqrt(computed_es.variance):.3f}")
print(f"95% CI: [{computed_es.effect_size - 1.96*np.sqrt(computed_es.variance):.3f}, "
      f"{computed_es.effect_size + 1.96*np.sqrt(computed_es.variance):.3f}]")

In [ ]:
# Example: Computing from t-statistic
t_value = 2.85
n1 = 25
n2 = 25

es_from_t = EffectSize.from_t_statistic(
    t=t_value,
    n1=n1,
    n2=n2,
    outcome_name="From t-stat"
)

print(f"Effect size from t={t_value}: d = {es_from_t.effect_size:.3f}")

## Step 3: Run Random-Effects Meta-Analysis

We'll use the DerSimonian-Laird estimator for between-study variance.

In [ ]:
# Initialize meta-analysis
try:
    ma = EffectSizeMetaAnalysis(dataset)
    print("Meta-analysis initialized successfully")
except ImportError as e:
    print(f"Error: {e}")
    print("\nInstall PyMARE with: pip install pymare")
    ma = None

In [ ]:
# Run analysis
if ma is not None:
    results = ma.run(
        method="DL",      # DerSimonian-Laird (most common)
        ci_level=0.95     # 95% confidence interval
    )
    
    print("\n" + "="*50)
    print("META-ANALYSIS RESULTS")
    print("="*50)
    print(f"\nCombined Effect Size (d): {results['combined_effect']:.3f}")
    print(f"95% CI: [{results['combined_ci'][0]:.3f}, {results['combined_ci'][1]:.3f}]")
    print(f"Z = {results['combined_z']:.2f}, p = {results['combined_pvalue']:.4f}")
    print(f"\nHeterogeneity:")
    print(f"  Q = {results['q_statistic']:.2f} (p = {results['q_pvalue']:.4f})")
    print(f"  I² = {results['i_squared']:.1f}%")
    print(f"  τ² = {results['tau_squared']:.4f}")
    print(f"  τ  = {results['tau']:.3f}")

In [ ]:
# Interpret I-squared
if ma is not None:
    i2 = results['i_squared']
    
    print("\nHeterogeneity Interpretation:")
    if i2 < 25:
        print(f"  I² = {i2:.1f}% indicates LOW heterogeneity")
        print("  Studies are relatively consistent")
    elif i2 < 50:
        print(f"  I² = {i2:.1f}% indicates MODERATE heterogeneity")
        print("  Some variation between studies")
    elif i2 < 75:
        print(f"  I² = {i2:.1f}% indicates SUBSTANTIAL heterogeneity")
        print("  Consider exploring moderators")
    else:
        print(f"  I² = {i2:.1f}% indicates CONSIDERABLE heterogeneity")
        print("  Strong variation - subgroup analysis recommended")

## Step 4: Forest Plot

Visualize individual study effects and the combined estimate.

In [ ]:
# Create forest plot
if ma is not None:
    fig = ma.forest_plot(
        title="Forest Plot: Intervention Effects on T-Maze Performance",
        effect_label="Cohen's d",
        show_weights=True
    )
    plt.show()

In [ ]:
# Custom forest plot with more control
if ma is not None:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    es_df = dataset.to_effect_sizes_df()
    n_studies = len(es_df)
    
    # Sort by effect size
    es_df = es_df.sort_values('effect_size', ascending=True).reset_index(drop=True)
    
    for i, row in es_df.iterrows():
        es = row['effect_size']
        se = np.sqrt(row['variance'])
        ci_low = es - 1.96 * se
        ci_high = es + 1.96 * se
        
        # CI line
        ax.hlines(i, ci_low, ci_high, colors='steelblue', linewidth=2)
        # Point
        ax.scatter(es, i, s=100, color='steelblue', zorder=3)
        # Label
        ax.text(-0.1, i, row['study_id'], ha='right', va='center', fontsize=10)
    
    # Combined effect
    combined = results['combined_effect']
    ci = results['combined_ci']
    ax.axvline(x=combined, color='red', linestyle='-', linewidth=2, alpha=0.7, label='Combined')
    ax.axvspan(ci[0], ci[1], alpha=0.2, color='red')
    
    # Null line
    ax.axvline(x=0, color='black', linestyle='--', linewidth=1)
    
    ax.set_yticks(range(n_studies))
    ax.set_yticklabels([])
    ax.set_xlabel("Cohen's d", fontsize=12)
    ax.set_title("Forest Plot (sorted by effect size)", fontsize=14)
    ax.legend(loc='lower right')
    
    plt.tight_layout()
    plt.show()

## Step 5: Funnel Plot & Publication Bias

Check for asymmetry that might indicate publication bias.

In [ ]:
# Create funnel plot
if ma is not None:
    fig, ax = plt.subplots(figsize=(8, 6))
    
    es_df = dataset.to_effect_sizes_df()
    
    effects = es_df['effect_size'].values
    se = np.sqrt(es_df['variance'].values)
    
    # Plot studies
    ax.scatter(effects, se, s=80, alpha=0.7, edgecolors='black')
    
    # Combined effect line
    combined = results['combined_effect']
    ax.axvline(x=combined, color='red', linestyle='-', linewidth=2, label='Combined effect')
    
    # Pseudo-confidence limits
    se_range = np.linspace(0.01, max(se) * 1.1, 100)
    ax.plot(combined - 1.96 * se_range, se_range, 'k--', alpha=0.5)
    ax.plot(combined + 1.96 * se_range, se_range, 'k--', alpha=0.5)
    
    # Fill the funnel
    ax.fill_betweenx(se_range, 
                     combined - 1.96 * se_range, 
                     combined + 1.96 * se_range,
                     alpha=0.1, color='gray')
    
    ax.set_xlabel("Effect Size (d)", fontsize=12)
    ax.set_ylabel("Standard Error", fontsize=12)
    ax.set_title("Funnel Plot", fontsize=14)
    ax.invert_yaxis()  # Larger studies at top
    ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    print("\nInterpretation:")
    print("- A symmetric funnel suggests no publication bias")
    print("- Asymmetry (e.g., missing studies in bottom-left) may indicate bias")

In [ ]:
# Egger's test for funnel plot asymmetry
from scipy import stats

if ma is not None:
    es_df = dataset.to_effect_sizes_df()
    
    effects = es_df['effect_size'].values
    se = np.sqrt(es_df['variance'].values)
    precision = 1 / se
    
    # Egger's regression: regress standardized effect on precision
    standardized = effects / se
    slope, intercept, r, p, stderr = stats.linregress(precision, standardized)
    
    print("Egger's Test for Publication Bias")
    print("="*40)
    print(f"Intercept: {intercept:.3f}")
    print(f"SE: {stderr:.3f}")
    print(f"t = {intercept/stderr:.2f}")
    print(f"p = {p:.4f}")
    print()
    if p < 0.05:
        print("Result: Significant asymmetry detected (p < 0.05)")
        print("Interpretation: Possible publication bias")
    else:
        print("Result: No significant asymmetry (p >= 0.05)")
        print("Interpretation: No strong evidence of publication bias")

## Step 6: Sensitivity Analysis

In [ ]:
# Leave-one-out analysis
if ma is not None:
    print("Leave-One-Out Sensitivity Analysis")
    print("="*50)
    
    es_df = dataset.to_effect_sizes_df()
    original_combined = results['combined_effect']
    
    loo_results = []
    
    for i in range(len(es_df)):
        # Create dataset without study i
        subset_effects = np.delete(es_df['effect_size'].values, i)
        subset_variance = np.delete(es_df['variance'].values, i)
        
        # Simple DL calculation
        w = 1 / subset_variance
        combined = np.sum(w * subset_effects) / np.sum(w)
        
        study_id = es_df.iloc[i]['study_id']
        change = combined - original_combined
        
        loo_results.append({
            'Excluded': study_id,
            'Combined_d': combined,
            'Change': change
        })
    
    loo_df = pd.DataFrame(loo_results)
    loo_df = loo_df.sort_values('Change', key=abs, ascending=False)
    
    print(f"\nOriginal combined effect: {original_combined:.3f}")
    print(f"\nWhen excluding each study:")
    print(loo_df.to_string(index=False))
    
    # Check if any exclusion changes conclusion
    max_change = loo_df['Change'].abs().max()
    print(f"\nMaximum change: {max_change:.3f}")
    print("Most influential study:", loo_df.iloc[0]['Excluded'])

## Step 7: Compare Methods

In [ ]:
# Compare different estimation methods
if ma is not None:
    methods = ["DL", "HE", "FE"]
    method_names = {
        "DL": "DerSimonian-Laird",
        "HE": "Hedges",
        "FE": "Fixed Effects"
    }
    
    print("Comparison of Estimation Methods")
    print("="*60)
    
    comparison = []
    for method in methods:
        try:
            r = ma.run(method=method)
            comparison.append({
                'Method': method_names[method],
                'Effect': r['combined_effect'],
                'CI_lower': r['combined_ci'][0],
                'CI_upper': r['combined_ci'][1],
                'tau2': r['tau_squared']
            })
        except Exception as e:
            print(f"  {method}: Error - {e}")
    
    comp_df = pd.DataFrame(comparison)
    print(comp_df.to_string(index=False))

## Step 8: Save Results

In [ ]:
# Create output directory
output_dir = Path("../results/effect_size")
output_dir.mkdir(parents=True, exist_ok=True)

if ma is not None:
    # Save forest plot
    fig = ma.forest_plot(title="Forest Plot")
    fig.savefig(output_dir / "forest_plot.png", dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"Saved forest plot to {output_dir / 'forest_plot.png'}")
    
    # Save effect sizes table
    es_df = dataset.to_effect_sizes_df()
    es_df.to_csv(output_dir / "effect_sizes.csv", index=False)
    print(f"Saved effect sizes to {output_dir / 'effect_sizes.csv'}")
    
    # Save summary
    summary = ma.summary()
    with open(output_dir / "summary.txt", "w") as f:
        f.write(summary)
    print(f"Saved summary to {output_dir / 'summary.txt'}")

In [ ]:
# Print final summary
if ma is not None:
    print(ma.summary())

## Conclusion

This notebook demonstrated:
1. Creating a dataset with effect sizes
2. Computing effect sizes from summary statistics
3. Running random-effects meta-analysis
4. Interpreting heterogeneity (I², Q, τ²)
5. Creating forest and funnel plots
6. Assessing publication bias
7. Sensitivity analysis

### Next Steps
- Add moderator analysis (meta-regression)
- Explore subgroup differences
- Combine with coordinate analysis from notebook 03